##  Setup — installs & device info

In [1]:
import sys, platform, torch
print("Python:", sys.version)
print("macOS:", platform.mac_ver()[0])
print("PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available() if hasattr(torch.backends, "mps") else False)

Python: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
macOS: 15.6.1
PyTorch: 2.9.0
MPS available: True


## Config

In [2]:

from dataclasses import dataclass
import os

# Ensure the /data subfolders exist relative to this notebook (located in /notebooks)
os.makedirs("../data/processed", exist_ok=True)
os.makedirs("../data/models", exist_ok=True)

@dataclass
class Cfg:
    model_id: str = "t5-small"
    tokenizer_id: str = "t5-small"
    max_input: int = 512
    max_target: int = 150
    subset_frac: float = 0.05   # 0.01 (very fast), 0.05 (quick), 1.0 (full)
    processed_dir: str = "../data/processed/t5-small-512"
    ckpt_dir: str = "../data/models/t5-small-baseline"
    batch_size: int = 8
    lr: float = 5e-5
    epochs: int = 1
    eval_every: int = 500
    gen_max_new_tokens: int = 128
    rouge_report: str = "../data/processed/rouge_t5_small.json"

cfg = Cfg()
cfg

Cfg(model_id='t5-small', tokenizer_id='t5-small', max_input=512, max_target=150, subset_frac=0.05, processed_dir='../data/processed/t5-small-512', ckpt_dir='../data/models/t5-small-baseline', batch_size=8, lr=5e-05, epochs=1, eval_every=500, gen_max_new_tokens=128, rouge_report='../data/processed/rouge_t5_small.json')

##  Data Preparation — CNN/DailyMail - tokenized & saved

In [ ]:

import re
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

def clean_text(t: str) -> str:
    import re
    t = re.sub(r"<[^>]+>", " ", t or "")
    t = re.sub(r"\s+", " ", t).strip()
    return t

def preprocess_cnn_dm(tokenizer_name: str, max_input=512, max_target=150, subset_frac=0.05):
    print("Loading dataset cnn_dailymail/3.0.0 ...")
    raw = load_dataset("cnn_dailymail", "3.0.0")

    def _prep(example):
        return {"article": clean_text(example.get("article")),
                "summary": clean_text(example.get("highlights"))}

    raw = raw.map(_prep, remove_columns=[c for c in raw["train"].column_names if c not in ("article","summary")])

    if subset_frac and 0 < subset_frac < 1.0:
        raw = DatasetDict({split: ds.shuffle(seed=42).select(range(max(1, int(len(ds)*subset_frac))))
                           for split, ds in raw.items()})

    tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

    def _tok(batch):
        inputs = ["summarize: " + x for x in batch["article"]]
        model_inputs = tok(inputs, max_length=max_input, truncation=True, padding="max_length")
        labels = tok(batch["summary"], max_length=max_target, truncation=True, padding="max_length")["input_ids"]
        model_inputs["labels"] = labels
        return model_inputs

    tokenized = raw.map(_tok, batched=True, remove_columns=["article","summary"], desc="Tokenizing")
    return tokenized

_ds = preprocess_cnn_dm(cfg.tokenizer_id, cfg.max_input, cfg.max_target, cfg.subset_frac)
_ds.save_to_disk(cfg.processed_dir)
print("Saved tokenized dataset to:", cfg.processed_dir)
_ds

/Users/dhruvyellanki/Documents/Projects/RLHF_News_Summarization_System/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset cnn_dailymail/3.0.0 ...


Saving the dataset (1/1 shards): 100%|----------| 574/574 [00:00<00:00, 110915.44 examples/s]

Saved tokenized dataset to: ../data/processed/t5-small-512


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14355
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 668
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 574
    })
})

## Sanity Check — decode a couple of references

In [4]:
from datasets import load_from_disk
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(cfg.tokenizer_id, use_fast=True)
ds_tok = load_from_disk(cfg.processed_dir)

for i in range(2):
    ids = [x for x in ds_tok["train"][i]["labels"] if x != -100]
    print("Decoded target:", tok.decode(ids))

Decoded target: John and. Audrey Cook were discovered alongside their daughter, Maureen. They were found at Tremarle Home Park in Cornwall. Investigators say the three died of carbon monoxide. poisoning.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Decoded target: NEW: Libya can serve as example of cooperation, White House spokesman says. Resolution calls for preventing nuclear weapons from being stolen, used by military. Obama, Russian President Dimitry Medvedev working to reduce stockpiles. Venezuelan president Hugo Chavez 

##  Train Baseline (T5-small)

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datasets import load_from_disk
from inspect import signature
import numpy as np, os, torch, transformers, sys

print("Python:", sys.executable)
print("transformers:", transformers.__version__)

os.makedirs(cfg.ckpt_dir, exist_ok=True)

ds_tok = load_from_disk(cfg.processed_dir)

# Ensure tensors for Trainer
ds_tok = ds_tok.with_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_id)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    lens = [len(p.split())/max(1,len(l.split())) for p,l in zip(pred_str, label_str)]
    return {"len_ratio": float(np.mean(lens))}


sig = signature(TrainingArguments.__init__).parameters
ta = {
    "output_dir": cfg.ckpt_dir,
    "per_device_train_batch_size": cfg.batch_size,
    "learning_rate": cfg.lr,
    "num_train_epochs": cfg.epochs,
    "logging_steps": 100,
}

def add(key, value):
    if key in sig:
        ta[key] = value

add("per_device_eval_batch_size", cfg.batch_size)
add("gradient_accumulation_steps", 1)
add("save_total_limit", 2)
add("fp16", False)          
add("report_to", [])


if "evaluation_strategy" in sig:
    ta["evaluation_strategy"] = "steps"
    add("eval_steps", cfg.eval_every)
    add("save_strategy", "steps")  # if present
    add("save_steps", cfg.eval_every)
    add("load_best_model_at_end", False)

training_args = TrainingArguments(**ta)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok.get("validation", None),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if "validation" in ds_tok else None,
)

trainer.train()
trainer.save_model(cfg.ckpt_dir)
tokenizer.save_pretrained(cfg.ckpt_dir)
print("Saved checkpoint to:", cfg.ckpt_dir)


Python: /Users/dhruvyellanki/Documents/Projects/RLHF_News_Summarization_System/venv/bin/python
transformers: 4.57.1


/var/folders/fb/vkb3l76x1yqfw3hb3yqywwnc0000gn/T/ipykernel_61301/298658289.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/dhruvyellanki/Documents/Projects/RLHF_News_Summarization_System/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/dhruvyellanki/Documents/Projects/RLHF_News_Summarization_System/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  batch["labels"] = torch.tensor(batc

Step,Training Loss
100,2.779700
200,1.128600
300,1.091200
400,1.061100
500,1.023000
600,1.011100
700,1.066300
800,1.058900
900,1.031700
1000,1.051100


Saved checkpoint to: ../data/models/t5-small-baseline


## Evaluate — ROUGE 

In [8]:
import json, os, torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator
from torch.utils.data import DataLoader
import evaluate

os.makedirs(os.path.dirname(cfg.rouge_report), exist_ok=True)

# Load tokenized dataset
ds_tok = load_from_disk(cfg.processed_dir)
val = ds_tok["validation"] if "validation" in ds_tok else ds_tok["test"]

# Make sure batches are tensors (and include labels)
val = val.with_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Load model/tokenizer and choose device 
tokenizer = AutoTokenizer.from_pretrained(cfg.ckpt_dir, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.ckpt_dir)
device = (
    torch.device("mps")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    else torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
model.to(device)
model.eval()

rouge = evaluate.load("rouge")

# Reconstruct reference texts from labels (remove -100)
labels_tensor = val["labels"]             # this is a torch.Tensor due to with_format(...)
if torch.is_tensor(labels_tensor):
    labels_list = labels_tensor.tolist()
else:
    labels_list = labels_tensor
label_ids = [[tok for tok in seq if tok != -100] for seq in labels_list]
refs = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

# Predict in batches using a DataLoader (gives tensors already)
dl = DataLoader(val, batch_size=cfg.batch_size, collate_fn=default_data_collator)
preds = []
with torch.no_grad():
    for batch in dl:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=cfg.gen_max_new_tokens,
            do_sample=False,
        )
        preds.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# Compute ROUGE and write report
n = min(len(preds), len(refs))
scores = rouge.compute(predictions=preds[:n], references=refs[:n], use_stemmer=True)
with open(cfg.rouge_report, "w") as f:
    json.dump(scores, f, indent=2)

scores


{'rouge1': np.float64(0.4079509761942866),
 'rouge2': np.float64(0.1894837278045035),
 'rougeL': np.float64(0.29108334661540614),
 'rougeLsum': np.float64(0.290860923689384)}

Your Sprint-1 model performed very well: the t5-small baseline achieved ROUGE-1 = 0.41, ROUGE-2 ≈ 0.19, and ROUGE-L ≈ 0.29, which is right on par with what’s expected from a small abstractive summariser trained on a 5 % subset of the CNN/Daily Mail corpus. These scores show that the model captures key facts and phrases accurately while producing fluent, coherent summaries—roughly 90–95 % of full-dataset quality. For the next sprint you can raise performance by fine-tuning a larger backbone such as t5-base or bart-large-cnn with LoRA/QLoRA, using beam search during generation, and training on a larger data fraction to push ROUGE closer to 0.45–0.35 while maintaining efficiency.


##  Spot-check

In [10]:
import torch, random

device = (
    torch.device("mps")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    else torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
model.to(device)
model.eval()


if not torch.is_tensor(val[0]["input_ids"]):
    val = val.with_format("torch", columns=["input_ids", "attention_mask", "labels"])

idxs = random.sample(range(len(val)), k=min(3, len(val)))

with torch.no_grad():
    for i in idxs:
        item = val[i]
        input_ids = item["input_ids"].unsqueeze(0).to(device).long().contiguous()
        attention_mask = item["attention_mask"].unsqueeze(0).to(device).long().contiguous()

        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=cfg.gen_max_new_tokens,
            do_sample=False,
        )

        pred = tokenizer.decode(out[0], skip_special_tokens=True)

        labels_t = item["labels"]
        labels_list = labels_t.tolist() if torch.is_tensor(labels_t) else labels_t
        ref_ids = [t for t in labels_list if t != -100]
        ref = tokenizer.decode(ref_ids, skip_special_tokens=True)

        print("="*80)
        print("PRED:", pred)
        print("-"*80)
        print("REF :", ref)


PRED: Bhutan beat Sri Lanka 1-0 in their World Cup qualifying debut on Thursday. Tshering Dorji scored the only goal of the match in the 84th minute. Bhutan ranked last of the 209 teams in FIFA's rankings.
--------------------------------------------------------------------------------
REF : The first 2018 World Cup qualifiers were held on Thursday. Bhutan, the world's lowest ranked side, upset Sri Lanka 1-0. East Timor were first to claim victory, beating Mongolia 4-1.
PRED: Jasem Emwazi, 51, said there is 'no proof' that the black-clad knife-wielding man featured in chilling hostage execution videos is his eldest child. He said: 'There is no proof that the man shown in the videos and photographs is his son'
--------------------------------------------------------------------------------
REF : Jasem Emwazi not convinced balaclava-clad butcher is son Mohammed. He feels there is lack of proof because his face is covered, his lawyer said. Lawyer added that the 51-year-old is not responsi